In [14]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk import everygrams

import re
import random
from collections import Counter, defaultdict
from copy import copy


POSSIBLE_CHARS = 'абвгдежзийклмнопрстуфхцчшщъыьэюя '

## Baseline

In [15]:
tokenizer = RegexpTokenizer(r'\w+')

In [33]:
with open('corpora/WarAndPeace.txt', 'r') as file:
    corpus_war_and_peace = file.readlines()

with open('corpora/AnnaKarenina.txt' ,'r') as file:
    corpus_karenina = file.readlines()

In [17]:
corpus = corpus_karenina + corpus_war_and_peace

In [18]:
corpus[1000:1010]

['– Да, – сказал Левин медленно и взволнованно. – Ты прав, я дик. Но только дикость моя не в том, что я уехал, а в том, что я теперь приехал. Теперь я приехал.\n',
 '\n',
 '– О, какой ты счастливец! – подхватил Степан Аркадьич, глядя в глаза Левину.\n',
 '\n',
 '– Отчего?\n',
 '\n',
 '– Узнаю коней ретивых по каким-то их таврам[47], юношей влюбленных узнаю по их глазам, – продекламировал Степан Аркадьич. – У тебя все впереди.\n',
 '\n',
 '– А у тебя разве уж назади?\n',
 '\n']

In [19]:
def tokenize(text, tokenizer=tokenizer):
    text = text.lower()
    res = []
    for symbol in text:
        if symbol in POSSIBLE_CHARS:
            res.append(symbol)
    text = ''.join(res)
    tokenized_text = tokenizer.tokenize(text)
    
    return ' '.join(tokenized_text)

In [20]:
def calc_freqs(text, n_gram=1, min_freq=0):
    
    if n_gram > 1:
        text = [''.join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)]
        
    res = {}
    counter_text = Counter(text)
    
    for key, value in counter_text.items():
        if value / len(text) > min_freq:
            res[key] = value / len(text)
            
    return res

In [22]:
corpus_tokenized = tokenize(' '.join(corpus))
freqs = calc_freqs(corpus_tokenized)

### Calc replacements

In [29]:
chars = list(freqs.keys())

replacements = np.random.choice(words, replace=False, size=(len(freqs, )))

char2replace = {}

for origin, replace in zip(chars, replacements):
    char2replace[origin] = replace

In [31]:
def get_reverse_mapping(corpus_freqs, text_freqs):
    
    # get ranks
    corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
    text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)
    
    reverse_dict = {}
    for text_char, text_freq in text_freqs_sorted:
        min_diff = 1
        best_char = None
        for corpus_char, corpus_freq in corpus_freqs_sorted:
            diff = abs(corpus_freq - text_freq)
            if diff < min_diff:
                best_char = corpus_char
                min_diff = diff

        reverse_dict[text_char] = best_char
        corpus_freqs_sorted = [(char, freq) for char, freq in corpus_freqs_sorted if char != best_char]
        
    return reverse_dict


In [58]:
def accuracy_with_chars(text1, text2):
    intersection = 0
    
    assert len(text1) == len(text2)
    
    for c1, c2 in zip(text1, text2):
        if c1 == c2:
            intersection += 1
    return intersection / len(text1)

In [46]:
sample_text = ['— Леди и джентльмены, — прозвучал голос из радиоприемника — четкий, спокойно-неумолимый мужской голос, совсем непохожий на те, что звучали в эфире уже много лет, — мистер Томпсон сегодня не будет говорить с вами. Его время истекло. С этого момента время принадлежит мне. Вы собирались выслушать сообщение о глобальном кризисе. Именно его вы сейчас и выслушаете. Три человека одновременно ахнули, узнав этот голос, но в криках толпы никто уже не мог услышать их, потому что крики толпы были оглушительны. Один из негромких возгласов выражал торжество, другой ужас, третий изумление. Три человека узнали говорившего: Дэгни, доктор Стадлер и Эдди Виллерс. Никто не смотрел на Эдди Виллерса, но Дэгни и доктор Стадлер посмотрели друг на друга. Она увидела, что его лицо исказилось от нестерпимого страха, он понял, что она знает и что она смотрит на него так, будто этот спокойный голос ударил его по лицу.']

In [48]:
print(sample_text)

['— Леди и джентльмены, — прозвучал голос из радиоприемника — четкий, спокойно-неумолимый мужской голос, совсем непохожий на те, что звучали в эфире уже много лет, — мистер Томпсон сегодня не будет говорить с вами. Его время истекло. С этого момента время принадлежит мне. Вы собирались выслушать сообщение о глобальном кризисе. Именно его вы сейчас и выслушаете. Три человека одновременно ахнули, узнав этот голос, но в криках толпы никто уже не мог услышать их, потому что крики толпы были оглушительны. Один из негромких возгласов выражал торжество, другой ужас, третий изумление. Три человека узнали говорившего: Дэгни, доктор Стадлер и Эдди Виллерс. Никто не смотрел на Эдди Виллерса, но Дэгни и доктор Стадлер посмотрели друг на друга. Она увидела, что его лицо исказилось от нестерпимого страха, он понял, что она знает и что она смотрит на него так, будто этот спокойный голос ударил его по лицу.']


In [49]:
tokenized_text = tokenize(' '.join(sample_text))

encoded_text = ''.join([char2replace.get(c, 'ь') for c in tokenized_text])
text_freqs = calc_freqs(encoded_text)

In [50]:
print(tokenized_text)

леди и джентльмены прозвучал голос из радиоприемника четкий спокойнонеумолимый мужской голос совсем непохожий на те что звучали в эфире уже много лет мистер томпсон сегодня не будет говорить с вами его время истекло с этого момента время принадлежит мне вы собирались выслушать сообщение о глобальном кризисе именно его вы сейчас и выслушаете три человека одновременно ахнули узнав этот голос но в криках толпы никто уже не мог услышать их потому что крики толпы были оглушительны один из негромких возгласов выражал торжество другой ужас третий изумление три человека узнали говорившего дэгни доктор стадлер и эдди виллерс никто не смотрел на эдди виллерса но дэгни и доктор стадлер посмотрели друг на друга она увидела что его лицо исказилось от нестерпимого страха он понял что она знает и что она смотрит на него так будто этот спокойный голос ударил его по лицу


In [51]:
print(text_freqs)

{'и': 0.050808314087759814, 'ь': 0.06928406466512702, 'л': 0.02771362586605081, 'ч': 0.07043879907621248, 'я': 0.15935334872979215, 'д': 0.010392609699769052, 'ж': 0.05542725173210162, 'с': 0.057736720554272515, 'щ': 0.009237875288683603, 'з': 0.03002309468822171, 'ю': 0.015011547344110854, 'р': 0.016166281755196306, 'а': 0.04157043879907621, 'т': 0.10969976905311778, 'ш': 0.012702078521939953, 'б': 0.03002309468822171, 'э': 0.02771362586605081, 'в': 0.012702078521939953, 'ы': 0.04734411085450346, 'ф': 0.03002309468822171, 'х': 0.04387990762124711, ' ': 0.023094688221709007, 'н': 0.011547344110854504, 'й': 0.006928406466512702, 'г': 0.009237875288683603, 'м': 0.0011547344110854503, 'е': 0.004618937644341801, 'о': 0.006928406466512702, 'у': 0.005773672055427252, 'п': 0.0011547344110854503, 'к': 0.0023094688221709007}


In [52]:
print(encoded_text)

иьлчячялдьжсищзьжюяратшбэвыияфтитхячшяаылчтрачьзжч ыявьс чняхрт тнжтжьэзтичзюнязэдх тняфтитхяхтбхьзяжьртйтдчняжыясьявстяшбэвыичябягмчаьяэдьязжтфтяиьсязчхсьаястзрхтжяхьфтлжеяжьяоэльсяфтбтачсщяхябызчяьфтябаьзеячхсь итяхягстфтязтзьжсыябаьзеярачжылиьдчсязжьябюяхточаыичхщябюхиэуысщяхттопьжчьятяфитоыищжтзя ачшчхьячзьжжтяьфтябюяхьнвыхячябюхиэуыьсьясачявьитбь ыятлжтбаьзьжжтяыйжэичяэшжыбягстсяфтитхяжтябя ач ыйястирюяжч стяэдьяжьязтфяэхиюуысщячйяртстзэявстя ач чястирюяоюичятфиэучсьищжюятлчжячшяжьфатз чйябтшфиыхтбябюаыдыиястадьхсбтялаэфтняэдыхясаьсчнячшэзиьжчьясачявьитбь ыяэшжыичяфтбтачбуьфтялгфжчялт стаяхсылиьаячягллчябчииьахяжч стяжьяхзтсаьияжыягллчябчииьахыяжтялгфжчячялт стаяхсылиьаяртхзтсаьичялаэфяжыялаэфыятжыяэбчльиыявстяьфтяичктячх ышчитхщятсяжьхсьарчзтфтяхсаыйыятжяртжеиявстятжыяшжыьсячявстятжыяхзтсачсяжыяжьфтясы яоэлстягстсяхрт тнжюняфтитхяэлыачияьфтяртяичкэ


In [53]:
reverse_mapping = get_reverse_mapping(freqs, text_freqs)
decoded_text = ''.join([reverse_mapping.get(c, 'ь') for c in encoded_text])

In [54]:
decoded_text

'таме е мйаинтшкаиб ьвочрузст дотол еч всмеоьвеакиепс занпеж льопожиоиаукотекбж куйлпож дотол лорлак иаьоюойеж ис на зно чрузсте р хъева уйа киодо тан келнав нокьлои ладомищ иа эуман доровенш л рске адо рвакщ елнапто л хнодо кокаинс рвакщ ьвеисмтайен киа рб лоэевстелш рблтуцснш лооэыаиеа о дтоэстшиок пвечела екаиио адо рб лажзсл е рблтуцсана нве заторапс омиорвакаиио сюиуте учиср хнон дотол ио р пвепсю нотьб иепно уйа иа код ултбцснш ею ьоноку зно пвепе нотьб эбте одтуценатшиб омеи еч иадвокпею рочдтслор рбвсйст новйалнро мвудож уйсл нванеж ечуктаиеа нве заторапс учисте дороверцадо мхдие мопнов лнсмтав е хмме реттавл иепно иа лконват ис хмме реттавлс ио мхдие е мопнов лнсмтав ьолконвате мвуд ис мвудс оис урематс зно адо тефо елпсчетолш он иалнавьекодо лнвсюс ои ьоищт зно оис чисан е зно оис лконвен ис иадо нсп эумно хнон льопожибж дотол умсвет адо ьо тефу'

In [61]:
accuracy_with_chars(decoded_text, tokenized_text)

0.2967667436489607

~ 30% accuracy

## Apply bigrams

In [64]:
def get_reverse_mapping_for_bigram(corpus_freqs, text_freqs, init_mapping=None):

    if not init_mapping:
        init_mapping = []
    reverse_mapping = {k: v for k, v in init_mapping}
    
    corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
    text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)
    
    
    for i, (text_bigram, text_freq) in enumerate(text_freqs_sorted):

        filtred_freqs = copy(corpus_freqs_sorted)
        
        if text_bigram[0] in reverse_mapping.keys():
            filtred_freqs = [(bigram, freq) for bigram, freq in filtred_freqs if bigram[0] == reverse_mapping[text_bigram[0]]]
            
        if text_bigram[1] in reverse_mapping.keys():
            filtred_freqs = [(bigram, freq) for bigram, freq in filtred_freqs if bigram[1] == reverse_mapping[text_bigram[1]]]
              
        min_diff = 1
        best_bigram = None
        for bigram, freq in filtred_freqs:
            diff = abs(freq - text_freq)
            if diff < min_diff:
                best_bigram = bigram
                min_diff = diff
                
        if text_bigram[0] not in reverse_mapping.keys():
            reverse_mapping[text_bigram[0]] = best_bigram[0]
            
        if text_bigram[1] not in reverse_mapping.keys():
            reverse_mapping[text_bigram[1]] = best_bigram[1]

        
    return reverse_mapping

In [63]:
corpus_freqs_bigram = calc_freqs(corpus_tokenized, n_gram=2)
text_freqs_bigram = calc_freqs(encoded_text, n_gram=2)


In [66]:
reverse_mapping_bigram = get_reverse_mapping_for_bigram(corpus_freqs_bigram, text_freqs_bigram)

In [67]:
decoded_text = ''.join([reverse_mapping_bigram.get(c, 'ь') for c in encoded_text])

In [68]:
print(decoded_text)

няко о ксястнвпясм п огвккен тонос ог  екооп ояпсоле кятлол спололсосякпонопмл пксслол тонос совсяп сяпогосол се тя кто гвккено в бро я кся псото нят постя  топпсос сятоксю ся лккят тово отв с вепо ято в япю остялно с бтото попясте в япю п осекнясот пся вм соло еносв вмснкиетв соолуясоя о тноленвсоп л огося опяссо ято вм сялкес о вмснкиеятя т о кяновяле оксов япяссо егскно кгсев бтот тонос со в л олег тонпм солто кся ся пот кснмиетв ог потопк кто л оло тонпм лмно отнкиотянвсм окос ог сят оплог вогтнесов вм есен то сяство к ктол ксес т ятол огкпнясоя т о кяновяле кгсено тово овиято кбтсо колто  стекня  о бкко воння с солто ся спот ян се бкко воння се со кбтсо о колто  стекня  поспот яно к кт се к кте осе квокяне кто ято ноео ослегоносв от сястя попото ст еге ос посюн кто осе гсеят о кто осе спот от се сято тел лккто бтот спололсмл тонос кке он ято по ноек


In [69]:
accuracy_with_chars(decoded_text, tokenized_text)

0.4168591224018476

~ 40 % accuracy

## MCMC sampling with bigrams

In [75]:
enumerate_chars = {c: i for i, c in enumerate(POSSIBLE_CHARS)}

In [76]:
def get_transitions_matrix(text, matrix_of_transitions):
    for i in range(len(text)-1):
        matrix_of_transitions[enumerate_chars[text[i]], enumerate_chars[text[i+1]]] += 1
    matrix_of_transitions = np.clip(matrix_of_transitions, 1, None)
    matrix_of_transitions = (np.log(matrix_of_transitions).T - np.log(matrix_of_transitions.sum(axis=1))).T
    return matrix_of_transitions

In [77]:
def calculate_log_likelihood(text, permutation):
    text = text.translate(str.maketrans(POSSIBLE_CHARS, ''.join(permutation)))
    return sum([matrix_of_transitions[enumerate_chars[text[i]], enumerate_chars[text[i+1]]] for i in range(len(text) - 1)])

In [81]:
from tqdm import tqdm

def decode_mcmc(text, iterations=10000):
    permutation = np.array(list(POSSIBLE_CHARS))
    random.shuffle(permutation)
    log_likelihood = calculate_log_likelihood(text, permutation)
    
    log_likelihood_best = log_likelihood
    permutation_best = permutation.copy()
    
    for i in tqdm(range(iterations)):
        swap = random.sample(range(len(POSSIBLE_CHARS)), 2)
        permutation[swap[0]], permutation[swap[1]] = permutation[swap[1]], permutation[swap[0]]
        log_likelihood_new = calculate_log_likelihood(text, permutation)
        if log_likelihood_new >= log_likelihood:
            log_likelihood = log_likelihood_new
            if log_likelihood_new > log_likelihood_best:
                log_likelihood_best = log_likelihood_new
                permutation_best = permutation.copy()
        else:
            if random.random() < np.exp(log_likelihood_new - log_likelihood):
                log_likelihood = log_likelihood_new
            else:
                permutation[swap[0]], permutation[swap[1]] = permutation[swap[1]], permutation[swap[0]]
    return text.translate(str.maketrans(POSSIBLE_CHARS, ''.join(permutation_best)))

In [ ]:
sample_text

In [85]:
matrix_of_transitions = get_transitions_matrix(corpus_tokenized, np.zeros((len(POSSIBLE_CHARS), len(POSSIBLE_CHARS))))
decoded = decode_mcmc(tokenized_text, 100000)




100%|██████████| 100000/100000 [00:57<00:00, 1751.55it/s]


In [86]:
accuracy_with_chars(decoded, tokenized_text)

1.0

bingo !

## Decode weird encodings

In [88]:
message = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'
encoded_characters = ''.join(set(message))
print(len(set(encoded_characters)))


28


In [92]:
top_freq = sorted(list(freqs.items()), key=lambda x: x[1], reverse=True)
top_freq_chars = [x[0] for x in top_freq]

In [94]:
rus_top_frequent_chars = ''.join(top_freq_chars)[:len(set(message))]
rus_top_frequent_chars

' оеанитслвркдмупяьгыбзчжйшхю'

In [ ]:
russian_part = str.maketrans(encoded_characters, rus_top_frequent_chars)
message = message.translate(russian_part)


In [97]:
print(message)

сраьгпигпьуьбсгялхотадяиыгьаьгмл бьгялхотадяиыгбснрбгйгшблюлгрллжесяьзгнлблхиыгасюнлгмхл ьбтбдгрнлхссгпрсюлгпигпрсгрусатаьгмхтпьадялгьгмлай ьбсготнрьотадяиыгжтаагчтгмлрасуяссг сбпсхблсгчтутяьсгнйхртгклбзгнляс ялгзгяь сюлгясглжсетв


In [99]:
decoded = decode_mcmc(message, 100000)

100%|██████████| 100000/100000 [00:16<00:00, 5933.33it/s]


In [101]:
decoded

'если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж'